In [ ]:

import ssl

import websocket
import json


def on_message(ws, message):
    data = json.loads(message)
    print(f"Received message: {data}")


def on_error(ws, error):
    print(f"Error: {error}")


def on_close(ws, close_status_code, close_msg):
    print(f"WebSocket closed with code: {close_status_code}, message: {close_msg}")


def on_open(ws):
    print("WebSocket connection opened")


if __name__ == "__main__":
    # Enable WebSocket trace logging
    websocket.enableTrace(True)

    ws = websocket.WebSocketApp(
        "wss://stream.binance.com:9443/ws/btcusdt@trade",
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close
    )

    # Suppress SSL certificate verification (Not recommended for production)
    ws.run_forever(sslopt={"cert_reqs": ssl.CERT_NONE})

In [1]:
!pip install websocket-client


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 141.1 kB/s eta 0:00:0000:0100:29
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for py4j==0.10.9.7 from https://files.pythonhosted.org/packages/10/30/a58b32568f1623aaad7db22aa9eafc4c6c194b429ff35bdc55ca2726da47/py4j-0.10.9.7-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 314.3 kB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=ee5a3300f2d21404cccee5eb656e1ac487ea3d15ef592939590fd1a3337a4018
  Stored in directory: /Users/andriy/Library/Caches/pip/wheels/07/a0/a3/d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [9]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("LocalSparkExample") \
    .master("local[1]") \
    .getOrCreate()

# Example operation
data = [("Alice", 29), ("Bob", 31), ("Cathy", 24)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)

df.show()

# Stop the session
spark.stop()

24/12/06 11:42:00 WARN Utils: Your hostname, Tarass-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.10.246.174 instead (on interface en0)
24/12/06 11:42:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/06 11:42:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----+---+
| Name|Age|
+-----+---+
|Alice| 29|
|  Bob| 31|
|Cathy| 24|
+-----+---+



In [ ]:
import websocket
import json
from queue import Queue
import threading

# Thread-safe queue
message_queue = Queue()

def on_message(ws, message):
    data = json.loads(message)
    message_queue.put(data)

def on_error(ws, error):
    print(f"WebSocket error: {error}")

def on_close(ws, close_status_code, close_msg):
    print(f"WebSocket closed with code: {close_status_code}, message: {close_msg}")

def on_open(ws):
    print("WebSocket connection opened")

    
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, FloatType, LongType, StructField
import json
import time
from queue import Queue

# Define schema for incoming WebSocket messages
schema = StructType([
    StructField("e", StringType(), True),  # Event type
    StructField("E", StringType(), True),   # Event time
    StructField("s", StringType(), True), # Symbol
    StructField("t", StringType(), True),   # Trade ID
    StructField("p", StringType(), True),  # Price
    StructField("q", StringType(), True),  # Quantity
    StructField("b", StringType(), True),   # Buyer order ID
    StructField("a", StringType(), True),   # Seller order ID
    StructField("T", StringType(), True),   # Trade time
    StructField("m", StringType(), True), # Is buyer the market maker?
    StructField("M", StringType(), True)  # Ignore
])

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("WebSocket Stream Processing") \
    .getOrCreate()

# Empty DataFrame to start
df = spark.createDataFrame([], schema)

# Thread-safe queue from WebSocket client
message_queue = Queue()

def update_dataframe(queue):
    global df
    batch = []
    while not queue.empty():
        message = queue.get()
        batch.append(message)
    if batch:
        # Create a new DataFrame and union with the existing one
        new_df = spark.createDataFrame(batch, schema=schema)
        df = df.union(new_df)
        df.show(truncate=False)

# Poll queue periodically and update DataFrame
ws = websocket.WebSocketApp(
        "wss://stream.binance.com:9443/ws/btcusdt@trade",
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close
    )
threading.Thread(target=ws.run_forever, kwargs={"sslopt": {"cert_reqs": ssl.CERT_NONE}}).start()
while True:
    update_dataframe(message_queue)
    time.sleep(1)

In [25]:
!pip install kafka-python

In [ ]:
from kafka import KafkaProducer, KafkaConsumer

# Connect to Kafka broker
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# Produce a message
producer.send('my_topic', value=b'Hello Kafka')

# Connect to Kafka consumer
consumer = KafkaConsumer('my_topic', bootstrap_servers='localhost:9092')

# Consume messages
for msg in consumer:
    print(msg.value)

In [18]:
df.show()

+---+---+---+---+---+---+---+---+---+---+---+
|  e|  E|  s|  t|  p|  q|  b|  a|  T|  m|  M|
+---+---+---+---+---+---+---+---+---+---+---+
+---+---+---+---+---+---+---+---+---+---+---+

